In [1]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation, load_step_function

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
nli_model = NLIModel(hg_model_hub_name, device='cuda')

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model size: 355,362,819


In [3]:
# MAIN GOAL
plan = """
IF your goal is to find a(n) non-living thing by moving it to the yellow box in the workshop THEN
move to kitchen,
pick up thermometer,
focus on thermometer,
move to workshop,
move thermometer to yellow box
"""

all_plans = [plan]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans)  # load plans from strings above
pl.load_plans_from_file("plans_navigation.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['find a(n) non-living thing by moving it to the yellow box in the workshop', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [4]:
env = ScienceWorldEnv("", "", envStepLimit=100)


task = 'find-non-living-thing'
env.load(task, 0)
randVariationIdx = env.getRandomVariationTest()
env.load(task, 28)

#goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()
goal = env.getTaskDescription().replace(". First, focus on the thing. Then,", "").replace("move", "by moving").replace("Your task is to", "").replace(".","").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])

goal

Task Name: boil variation 285
Task Description: Your task is to find a(n) non-living thing. First, focus on the thing. Then, move it to the yellow box in the workshop.


'find a(n) non-living thing by moving it to the yellow box in the workshop'

In [5]:
step_function = load_step_function(env, goal)


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.score)

find a(n) non-living thing by moving it to the yellow box in the workshop
move to kitchen
move to workshop
Completed keys: 
----------------------------------------------------------------------------------------------------
Sequential Subgoals:
----------------------------------------------------------------------------------------------------
0	true	               GoalFocusOnNonlivingThing	focus on a non-living thing
1	true	                   GoalObjectInContainer	move living thing to answer box
----------------------------------------------------------------------------------------------------
Unordered and Optional Subgoals:
----------------------------------------------------------------------------------------------------
0	true	                   GoalObjectInContainer	Pick up object
1	true	                  GoalInRoomWithOpenDoor	Be in a room with an open door
2	true	                   GoalMoveToNewLocation	Move to a new location
3	true	                      GoalMoveToLocation	M